<a href="https://colab.research.google.com/github/dhnanjay/Notebook2/blob/master/Summarize_a_PDF_Document.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#https://pspdfkit.com/blog/2022/summarize-pdf-document-using-machine-learning-and-natural-language-processing/
#https://colab.research.google.com/drive/1E9H6ra3kXOc8U1hfamRA7IUJjQj9uMAg?usp=sharing


In [1]:
!pip install Sentencepiece
!pip install pdfplumber transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Upload a PDF document from your local drive.

In [2]:
from google.colab import files
uploaded = files.upload()

Saving abb_2022.pdf to abb_2022.pdf


Import the document.
Make sure the filename matches the name of the uploaded file.

In [3]:
import pdfplumber

with pdfplumber.open(r'abb_2022.pdf') as pdf:
  extracted_page = pdf.pages[111]
  extracted_text = extracted_page.extract_text()
  print(extracted_text)


ABB INDIA LIMITED ANNUAL REPORT 2021 111
—
Annexure - D to Board’s Report
ANNUAL REPORT ON CORPORATE SOCIAL RESPONSIBILITIES (CSR) ACTIVITIES
1.  Brief outline on CSR Policy of the Company:
Corporate Social Responsibility Policy of ABB India Limited
ABB as a responsible Corporate entity, the Company strives to reach out to wider community to ensure well-being of 
needy as a part of its Corporate Social Responsibility (“CSR”). The Company  has always believed in and contributed to 
the society. Along with sustained economic performance, environmental and social stewardship is also a key factor for 
holistic business growth and is internalized as integral part of the Company’s Strategy.
ABB India views corporate social responsibility (CSR) as a channel to further its cause for a better world by writing the 
future together with its multiple stakeholders. The Company is committed to sustainable and inclusive development 
of the community’s social capital through active engagement. The Com

Use the `transformers` library offered by [Hugging Face](https://huggingface.co/), and use the Bart [tokenizer](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization) with a [distilled Bart model](https://huggingface.co/sshleifer/distilbart-cnn-12-6) specifically trained for text summarization.

In [31]:
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig

model = BartForConditionalGeneration.from_pretrained('sshleifer/distilbart-cnn-12-6')
tokenizer = BartTokenizer.from_pretrained('sshleifer/distilbart-cnn-12-6')

inputs = tokenizer([extracted_text], truncation=True, return_tensors='pt')

# Generate Summary
summary_ids = model.generate(inputs['input_ids'], num_beams=4, early_stopping=True, min_length=0, max_length=4096)
summarized_text = ([tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in summary_ids])


In [8]:
print(summarized_text[0])

 ABB India views corporate social responsibility (CSR) as a channel to further its cause for a better world by writing the future together with its multiple stakeholders. The Company’s CSR programs are aimed to contribute,  catalyze and augment Governmental efforts in the social development agenda, predominantly in areas where it operates.


In [11]:
print(summarized_text[0])

 ABB India views corporate social responsibility (CSR) as a channel to further its cause for a better world by writing the future together with its multiple stakeholders. The Company’s CSR programs are aimed to contribute,  catalyze and augment Governmental efforts in the social development agenda, predominantly in areas where it operates.


In [12]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

In [13]:
finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')


Downloading:   0%|          | 0.00/533 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/439M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [14]:
nlp = pipeline("sentiment-analysis", model=finbert, tokenizer=tokenizer)

sentences = ["there is a shortage of capital, and we need extra financing",  
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]
results = nlp(sentences)
print(results)

[{'label': 'Negative', 'score': 0.9966173768043518}, {'label': 'Positive', 'score': 1.0}, {'label': 'Negative', 'score': 0.9999710321426392}, {'label': 'Neutral', 'score': 0.9889441728591919}]


In [19]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, TFPegasusForConditionalGeneration

# Let's load the model and the tokenizer 
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name) # If you want to use the Tensorflow model 
                                                                    # just replace with TFPegasusForConditionalGeneration


# Some text to summarize here extracted_text
text_to_summarize = """ National Commercial Bank (NCB), Saudi Arabia’s largest lender by assets, agreed to 
buy rival Samba Financial Group for $15 billion in the biggest banking takeover this year.
NCB will pay 28.45 riyals ($7.58) for each Samba share, according to a statement on Sunday, 
valuing it at about 55.7 billion riyals. NCB will offer 0.739 new shares for each Samba share, 
at the lower end of the 0.736-0.787 ratio the banks set when they signed an initial framework agreement in June.
The offer is a 3.5% premium to Samba’s Oct. 8 closing price of 27.50 riyals and about 24% higher than the level the shares 
traded at before the talks were made public. Bloomberg News first reported the merger discussions.
The new bank will have total assets of more than $220 billion, creating the Gulf region’s third-largest lender. 
The entity’s $46 billion market capitalization nearly matches that of Qatar National Bank QPSC, 
which is still the Middle East’s biggest lender with about $268 billion of assets."""



In [30]:
# Tokenize our text
# If you want to run the code in Tensorflow, please remember to return the particular tensors as simply as using return_tensors = 'tf'
input_ids = tokenizer(extracted_text[:2500], return_tensors="pt").input_ids

# Generate the output (Here, we use beam search but you can also use any other strategy you like)
output = model.generate(
    input_ids, 
    max_length=1128, 
    num_beams=5, 
    early_stopping=False
)

# Finally, we can print the generated summary
print(tokenizer.decode(output[0], skip_special_tokens=True))
# Generated Output: Saudi bank to pay a 3.5% premium to Samba share price. Gulf region’s third-largest lender will have total assets of $220 billion


ABB India’s CSR policies and programs are aimed at contributing to the society.


In [21]:
len(text_to_summarize)

993

In [22]:
len(extracted_text)

3438

In [24]:
extracted_text[:993]

'ABB INDIA LIMITED ANNUAL REPORT 2021 111\n—\nAnnexure - D to Board’s Report\nANNUAL REPORT ON CORPORATE SOCIAL RESPONSIBILITIES (CSR) ACTIVITIES\n1.  Brief outline on CSR Policy of the Company:\nCorporate Social Responsibility Policy of ABB India Limited\nABB as a responsible Corporate entity, the Company strives to reach out to wider community to ensure well-being of \nneedy as a part of its Corporate Social Responsibility (“CSR”). The Company  has always believed in and contributed to \nthe society. Along with sustained economic performance, environmental and social stewardship is also a key factor for \nholistic business growth and is internalized as integral part of the Company’s Strategy.\nABB India views corporate social responsibility (CSR) as a channel to further its cause for a better world by writing the \nfuture together with its multiple stakeholders. The Company is committed to sustainable and inclusive development \nof the community’s social capital through active engage

In [33]:
import spacy

# load the English language model
nlp = spacy.load("en_core_web_sm")

# define the text corpus
text = "Mr. John Smith is the CEO of XYZ Company and can be reached at john.smith@xyz.com. Ms. Jane Doe is the CFO."

# process the text using the SpaCy library
doc = nlp(extracted_text)

# iterate over the entities in the text
for ent in doc.ents:
    # print the entity text and its label
    print(ent.text, ent.label_)

ABB INDIA ORG
2021 DATE
111 CARDINAL
Board’s Report ORG
CSR ORG
1 CARDINAL
CSR Policy ORG
Corporate Social Responsibility Policy ORG
ABB India Limited ORG
ABB ORG
Company ORG
Corporate Social Responsibility ORG
CSR ORG
ABB India ORG
CSR ORG
Company ORG
CSR ORG
Governmental ORG
Company’s ORG
CSR ORG
1 CARDINAL
ABB Value ‘Curiosity ORG
2 CARDINAL
Diversity and Inclusion ORG
ABB Value ‘Courage ORG
3 CARDINAL
Communities ORG
ABB Value ‘Care ORG
2 CARDINAL
Composition of CSR ORG
CSR ORG
CSR ORG
the year DATE
the year
1 Mrs Renu Sud Karnad               Chairperson DATE
Committee ORG
2 CARDINAL
Wierod PERSON
4 4 CARDINAL
Sanjeev Sharma PERSON
4 4 CARDINAL
4 4 CARDINAL
CSR Committee ORG
CSR Policy ORG
CSR ORG
4 CARDINAL
Impact  Assessment  of  CSR WORK_OF_ART
8 CARDINAL
2014 DATE
Third ORDINAL
CSR ORG
1 CARDINAL
200 CARDINAL
2 CARDINAL
Peenya GPE
3 CARDINAL
Agastya International Foundation ORG
4 CARDINAL
IIT ORG
Roorkee GPE
IIT ORG
Roorkee GPE
Uttarakhand GPE
Uttarakhand ORG
5 CARDINAL
3 CARD

In [34]:
import torch
from transformers import pipeline

summarizer = pipeline(
    "summarization",
    "pszemraj/long-t5-tglobal-base-16384-book-summary",
    device=0 if torch.cuda.is_available() else -1,
)


Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [35]:
long_text = "Here is a lot of text I don't want to read. Replace me"

result = summarizer(extracted_text)
print(result[0]["summary_text"])

/usr/local/lib/python3.8/dist-packages/transformers/modeling_utils.py:722: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Ab India's annual report on corporate social responsibility is now out. The company has set up a committee to review the progress of its CSR activities and make an annual report, which will be made publicly available after the financial year ends.


In [ ]:
Ab India's annual report on corporate social responsibility is now out. 
The company has set up a committee to review the progress of its CSR 
activities and make an annual report, which will be made publicly available after the financial year ends.




In [ ]:
ABB India views corporate social responsibility (CSR) as a channel to further its cause for a better world by writing 
the future together with its multiple stakeholders. The Company’s CSR programs are aimed to contribute,  
catalyze and augment Governmental efforts in the social development agenda, predominantly in areas where it operates.